In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data

In [ ]:
def get_tblogs(data_id):
    root_dir = "/home/ubuntu/ray_results/PPO"
    exp_path = glob.glob(os.path.join(root_dir, "*" + data_id + "*", "event*"))
    if len(exp_path) > 1:
        print("Error: more than one experiments with that id found")
    df = tflog2pandas(exp_path[0])
    df = df.pivot(index="step", columns="metric")
    df_mean = df.filter(regex=(".*(custom_metrics|reward).*_mean"))
    df_mean = df_mean.rename(columns={x: x[24:-5] if "custom_metrics" in x else x[9:-5] for _,x in df.columns})
    return df_mean

In [ ]:
def gen_plot_func(regex, title="", xlabel="", ylabel="", ylim=None, figure=True):
    def plot_func(df):
        if figure:
            plt.figure()
        plt.plot(df.filter(regex=(regex)))
        plt.ylim(ylim)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
    return plot_func

In [ ]:
def step_average(a, steps):
    b = np.zeros(a.shape)
    c = 0
    for i in range(len(a)):
        c += a[i]
        if i >= steps:
            c -= a[i-steps]
        b[i] = c / min(steps, i+1)
    return b

In [ ]:
df_rl = get_tblogs("9db51")

In [ ]:
df_alg = get_tblogs("23591")

In [ ]:
df_rl = df_rl[df_rl.index <= 2000000]
df_alg = df_alg[df_alg.index <= 2000000]

dfs = [df_rl, df_alg]

In [ ]:
list(map(gen_plot_func(".*cost/energy", ylim=[0, 0.03], figure=False, xlabel="Time [s]", ylabel="Power [W]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Cooling power")

In [ ]:
list(map(gen_plot_func(".*reward", ylim=[-30, 0], figure=False, xlabel="Time [s]", ylabel="Power [W]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Reward")

In [ ]:
list(map(gen_plot_func(".job/dropped", ylim=[0, 0.2], figure=False, xlabel="Time [s]", ylabel="Reward"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Fraction of jobs dropped")

In [ ]:
gen_plot_func(".*srv\d+/temp_cpu", xlabel="Time [s]", ylabel="Temperature [C]", title="CPU temperature")(dfs[0])

In [ ]:
list(map(gen_plot_func(".*srv\d+/load", xlabel="Time [s]", ylabel="Load [W]", title="Load distribution"), dfs));

In [ ]:
list(map(gen_plot_func(".crah0/temp_out", figure=False, xlabel="Time [s]", ylabel="Temperature [C]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("CRAH temperature setpoint")

In [ ]:
list(map(gen_plot_func(".crah0/flow", figure=False, xlabel="Time [s]", ylabel="Flow [$m^3/s$]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("CRAH flow setpoint")

In [ ]:
list(map(gen_plot_func(".srv/server_total_flow", figure=False, xlabel="Time [s]", ylabel="Flow [$m^3/s$]"), dfs))
plt.legend(["RL", "Constant"])
plt.title("Total flow through servers")